In [12]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import matplotlib as mpl

mpl.style.use("seaborn")

In [13]:
S = np.load("../data/S.npy")
P = np.load("../data/P.npy")
F = np.load("../data/F.npy")

molecules = np.load("../data/molecules.npy")

In [15]:
from SCFInitialGuess.utilities.dataset import Dataset

dim = 26

ind_cut = 150
index = np.arange(200)

S_test = np.array(S)[index[150:]]
P_test = np.array(P)[index[150:]]
F_test = np.array(F)[index[150:]]
molecules_test = [molecules[index[i]] for i in range(150, 200)]

S_train = np.array(S)[index[:150]]
P_train = np.array(P)[index[:150]]
F_train = np.array(F)[index[:150]]
molecules_train = [molecules[index[i]] for i in range(150)]

dataset = Dataset(np.array(S_train), np.array(F_train), split_test=0.0)

dataset.testing = (Dataset.normalize(S_test, mean=dataset.x_mean, std=dataset.x_std)[0], F_test)

[-] 2018-05-10 18:50:00: Data set normalized. Mean value std: 0.008732722987069362


In [48]:
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.training import Trainer, RegularizedMSE, MSE

trainer = Trainer(
    EluTrNNN([dim**2, 2 * dim**2, dim**2, dim**2]),
    cost_function=MSE(),
    #cost_function=RegularizedMSE(alpha=1e-7),
    optimizer=tf.train.AdamOptimizer(learning_rate=1e-3)
)

trainer.setup()
network, sess = trainer.train(
    dataset,
    convergence_threshold=1e-6
)
graph = trainer.graph

[-] 2018-05-10 19:19:08: No target graph specified for Trainer setup. Creating new graph ...
[-] 2018-05-10 19:19:08: Setting up the training in the target graph ...
[-] 2018-05-10 19:19:08: network ...
[-] 2018-05-10 19:19:08: error function ...
[-] 2018-05-10 19:19:08: cost function ...
[-] 2018-05-10 19:19:08: training step
[-] 2018-05-10 19:19:09: Starting network training ...
[ ] 2018-05-10 19:19:09: Val. Cost: 8.979E-01. Error: 8.979E-01. Diff: 1.0E+10
[ ] 2018-05-10 19:19:12: Val. Cost: 6.691E-04. Error: 6.691E-04. Diff: 9.0E-01
[ ] 2018-05-10 19:19:16: Val. Cost: 4.054E-04. Error: 4.054E-04. Diff: 2.6E-04
[ ] 2018-05-10 19:19:20: Val. Cost: 3.459E-04. Error: 3.459E-04. Diff: 6.0E-05
[ ] 2018-05-10 19:19:23: Val. Cost: 3.825E-04. Error: 3.825E-04. Diff: 3.7E-05
[ ] 2018-05-10 19:19:26: Val. Cost: 2.413E-04. Error: 2.413E-04. Diff: 1.4E-04
[ ] 2018-05-10 19:19:29: Val. Cost: 1.390E-04. Error: 1.390E-04. Diff: 1.0E-04
[ ] 2018-05-10 19:19:32: Val. Cost: 1.261E-04. Error: 1.261E-04

# Calculate density matrices 

In [49]:
from pyscf.scf import hf

def get_occ(mo_energy, mo_coeff, n_electrons = 30):
    
    
    e_idx = np.argsort(mo_energy)
    e_sort = mo_energy[e_idx]
    nmo = mo_energy.size
    mo_occ = np.zeros(nmo)
    nocc = n_electrons // 2
    mo_occ[e_idx[:nocc]] = 2
    
    return mo_occ

def make_density(s, f):
        mo_energy, mo_coeff = hf.eig(f.reshape(dim, dim), s.reshape(dim, dim))
        mo_occ = get_occ(mo_energy, mo_coeff)
        return hf.make_rdm1(mo_coeff, mo_occ)
    
with graph.as_default():
    p_f = np.array(
        [make_density(s, f).reshape(dim**2) for (s,f) in zip(S_test, network.run(sess, dataset.testing[0]))]
    )

# Analysis 

In [53]:
from SCFInitialGuess.utilities.analysis import measure_all_quantities, mf_initializer

n_electrons = 30

results = measure_all_quantities(
    p_f,
    dataset,
    molecules,
    n_electrons,
    mf_initializer,
    dim,
    is_triu=False    
)

Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs sys.args
Warn: Ipython shell catchs s

[ ] 2018-05-10 19:27:56: Iteration calculation: 0


Warn: Ipython shell catchs sys.args


[ ] 2018-05-10 19:27:56: Iteration calculation: 1


Warn: Ipython shell catchs sys.args


...

Warn: Ipython shell catchs sys.args


[ ] 2018-05-10 19:28:08: Iteration calculation: 48


Warn: Ipython shell catchs sys.args


[ ] 2018-05-10 19:28:08: Iteration calculation: 49


Warn: Ipython shell catchs sys.args


In [54]:
from SCFInitialGuess.utilities.analysis import make_results_str

print(make_results_str(results))

--- Absolute Error ---
3.18829E-01 +- 5.70472E-03
--- Symmetry Error ---
1.47883E-18 +- 3.02257E-19
--- Idempotence Error ---
3.08606E-16 +- 4.85346E-17
--- Occupance Error ---
7.31859E-15 +- 5.52261E-15
--- HF Energy Error ---
3.16903E+03 +- 3.77672E+01
--- Avg. Iterations ---
1.55800E+01 +- 2.35873E+00

